In [1]:
#AC typeTranslate test

import pandas as pd

from specklepy.api.wrapper import StreamWrapper
from specklepy.api.client import SpeckleClient
from specklepy.api import operations

from specklepy.objects import Base
from specklepy.objects.other import Collection
from specklepy.objects.geometry import Mesh
from specklepy.objects.other import RenderMaterial

from speckle_automate import (
    AutomateBase,
    AutomationContext,
    execute_automate_function,
)
######conversion DICT hard coded

cTable = {
    "Zone":"_Curves",
    "Wall":"_Walls_50",
    "Column":"_Columns_50",
    "Slab":"_Floors_20",
    "Stair":"_Stairs_20",
    "Ceiling":"_Ceilings_70",
    "Roof":"_Roof_20",
    "Railing":"_Railings_20",
    "Mesh":"_Ground_20",
    "Morph":"_Context_30",
    "CurtainWall":"_CurtainWall",
    "Window":"_Windows",
    "Door":"_Door"
    }


#####Functions here#######
#Send to stream
def Send(DL_modell, Stream):
    #sWrap = StreamWrapper ("https://multiconsult.speckle.xyz/projects/8c1aca44f6")
    sWrap = StreamWrapper (Stream)
    sTransport = sWrap.get_transport()
    sClient = sWrap.get_client()

    sendID = operations.send(DL_modell, [sTransport])
    sClient.commit.create(sWrap.stream_id, sendID, message= " simple translate test" )
def stripString(string):
    noSpaces = string.replace(" ", "")
    return noSpaces
def dupeStream(source, target = "https://multiconsult.speckle.xyz/projects/8c1aca44f6"):



    #receive source
    SarehACurl = source
    dupWrapper = StreamWrapper(SarehACurl)
    dupClient = dupWrapper.get_client()
    dupTrans = dupWrapper.get_transport()
    dupComm = dupClient.commit.get(dupWrapper.stream_id, dupWrapper.commit_id)
    dupObj = operations.receive(dupComm.referencedObject, dupTrans)

    #send copy of Stream
    sWrap = StreamWrapper (target)

    sTransport = sWrap.get_transport()
    sClient = sWrap.get_client()

    sendID = operations.send(dupObj, [sTransport])
    sClient.commit.create(sWrap.stream_id, sendID, message= f"duppped from soruce: {source}")

    print ("DUP COMPLETE")

    return
def kCheck(obj,key):
    ###check for attribute in Base
    try: 
        if key in obj.get_member_names():
            return True
        else:
            return False
    except:
        return False
    
def createUstrings(obj,list = ["level", "type", "layer", "height", "width", "thickness", "room", "lengh", "area", "name", "number","nummer","elemenetType"]):

    ###Attempt to add specific attributes as user strings 

    #list = ["level", "layer", "height", "thickness", "room", "lengh", "area", "name", "number","elemenetType"]
    #obj.userStrings = Base( level = obj.level.name, layer = str(obj.layer), applicationId = str(obj.applicationId), height = str(obj.height), thickness = str(obj.thickness), elementType = obj.elementType)
    userStrings = Base (appID = str(obj.applicationId), speckleType = str(obj.speckle_type))

    for i in list:
        if kCheck(obj,i) == True:
            if type(obj[i]) != type(Base()):
                userStrings[i] = str(obj[i])
            else:
                userStrings[i] = obj[i].name
    return userStrings

def getChildrenGlass(input):
    #get children elements from BaseObject
    #Takes Collection
    subEle = {}
    returnThis = {"ALT1":subEle, "ALT2":[]}

      
    if type(input) == Collection:
        objList = input.elements
        isCol = True
    elif type(input) != list:
        objList = [input]
    else:
        objList = input

    for obj in objList:
        if "elements" in obj.get_member_names() and obj.elements is not None:
            for j in obj.elements:
                
                parentStrings = createUstrings(j)
                
                if (j.elementType in cTable.keys()) and (cTable[j.elementType] not in [col.name for col in dl_Inputs]) and (j.displayValue is not None) :
                    for dMesh in j.displayValue:
                        dMesh.userStrings = parentStrings
                        ###ALT1: adding each dValue mesh at a time as a new object in collection elements. Worth trying to separate into one object for Solids and one Object for Glass?
                        if dMesh.renderMaterial.opacity < 1 and (cTable[stripString(j.elementType)] + ">Glass_70%") not in subEle.keys():
                            subEle[cTable[stripString(j.elementType)] + ">Glass_70%"] = (Collection(name = j.elementType + ">Glass_70%", elements = [dMesh], collectionType = "layer"))

                        elif dMesh.renderMaterial.opacity < 1 and (cTable[stripString(j.elementType)] + ">Glass_70%") in subEle.keys():
                            subEle[cTable[stripString(j.elementType)] + ">Glass_70%"].elements.append(dMesh)

                        elif dMesh.renderMaterial.opacity == 1 and (cTable[stripString(j.elementType)] + ">Solid_50") not in subEle.keys():
                            subEle[cTable[stripString(j.elementType)] + ">Solid_50"] = (Collection(name = j.elementType + ">Solid_50", elements = [dMesh], collectionType = "layer"))
                        
                        elif dMesh.renderMaterial.opacity == 1 and (cTable[stripString(j.elementType)] + ">Solid_50") in subEle.keys():
                            subEle[cTable[stripString(j.elementType)] + ">Solid_50"].elements.append(dMesh)

                        ###ALT2: Create dValueSolid base with all the opaque meshes and dValueGlass with all the transparent meshes
                        if dMesh.renderMaterial.opacity == 1:
                            if "dValueSolid" in j.get_member_names():
                                j.dValueSolid.append(dMesh)
                            else:
                                j.dValueSolid = [dMesh]
                        if dMesh.renderMaterial.opacity <1:
                            if "dValueGlass" in j.get_member_names():
                                j.dValueGlass.append(dMesh)
                            else:
                                j.dValueGlass = [dMesh]
                                
                returnThis["ALT2"].append(j)

    return returnThis

def getChildren(input):
    #get children elements from Base() / list of Base() / collection of Base()
    #returns one list
    subEle = {}
    returnThis = []

      
    if type(input) == Collection:
        objList = input.elements
        isCol = True
    elif type(input) != list:
        objList = [input]
    else:
        objList = input

    for obj in objList:
        if "elements" in obj.get_member_names() and obj.elements is not None:
            returnThis.extend(obj.elements)
            """for j in obj.elements:
                
                parentStrings = createUstrings(j)
                if (j.elementType in cTable.keys()) and (cTable[j.elementType] not in [col.name for col in dl_Inputs]) and (j.displayValue is not None) :
                    for dMesh in j.displayValue:
                        dMesh.userStrings = parentStrings
                        ###ALT1: adding each dValue mesh at a time as a new object in collection elements."""
    return returnThis        

def findGlass(input):

    #Takes Collection, List of Base() or Base()
    #format input to list
    if type(input) == Collection:
        objList = input.elements
        isCol = True
    elif type(input) != list:
        objList = [input]
    else:
        objList = input

    #Separate displayMeshes by opacity
    subEle = {}
    for j in objList:
        parentStrings = createUstrings(j)
            
        eleType = stripString(j.elementType)

        if (eleType in cTable.keys()) and (j.displayValue is not None) :
            colList = [col.name for col in dl_Inputs]

            if (cTable[eleType] in colList):

                index = colList.index(cTable[eleType])
                #dl_Inputs.pop(index)

            for dMesh in j.displayValue:
                dMesh.userStrings = parentStrings
                
                
                ###ALT1: adding each dValue mesh at a time as a new object in collection elements. Worth trying to separate into one object for Solids and one Object for Glass?
                if dMesh.renderMaterial.opacity < 1 and (cTable[eleType] + ">Glass_70%") not in subEle.keys():
                    subEle[cTable[eleType] + ">Glass_70%"] = (Collection(name = cTable[eleType] + ">Glass_70%", elements = [dMesh], collectionType = "layer"))

                elif dMesh.renderMaterial.opacity < 1 and (cTable[eleType] + ">Glass_70%") in subEle.keys():
                    subEle[cTable[eleType] + ">Glass_70%"].elements.append(dMesh)

                elif dMesh.renderMaterial.opacity == 1 and (cTable[eleType] + ">Solid_50") not in subEle.keys():
                    subEle[cTable[eleType] + ">Solid_50"] = (Collection(name = cTable[eleType] + ">Solid_50", elements = [dMesh], collectionType = "layer"))
                
                elif dMesh.renderMaterial.opacity == 1 and (cTable[eleType] + ">Solid_50") in subEle.keys():
                    subEle[cTable[eleType] + ">Solid_50"].elements.append(dMesh)

    return subEle

def translateModellAC(ACmodell):

    pass

    

In [46]:
###Receive AC Stream data
#urlStream = "https://multiconsult.speckle.xyz/projects/8c1aca44f6/models/8819271698@91c950d961"
urlStream = "https://multiconsult.speckle.xyz/projects/8c1aca44f6/models/8819271698@9621d4f8bb"
wrapper = StreamWrapper(urlStream)
client = wrapper.get_client()
transport = wrapper.get_transport()

comm = client.commit.get(wrapper.stream_id, wrapper.commit_id)
cData = operations.receive(comm.referencedObject, transport)


"""if comm.sourceApplication == "Archicad":
    print ("AC")
"""

AC_Collections = cData.elements

for i in AC_Collections:
    print (f"{i.name} - {i.speckle_type} !!!" )

Wall - Speckle.Core.Models.Collection !!!
Roof - Speckle.Core.Models.Collection !!!
Column - Speckle.Core.Models.Collection !!!
Beam - Speckle.Core.Models.Collection !!!
Object - Speckle.Core.Models.Collection !!!
GridElement - Speckle.Core.Models.Collection !!!
Slab - Speckle.Core.Models.Collection !!!
Zone - Speckle.Core.Models.Collection !!!
CurtainWall - Speckle.Core.Models.Collection !!!
Morph - Speckle.Core.Models.Collection !!!
Stair - Speckle.Core.Models.Collection !!!
Railing - Speckle.Core.Models.Collection !!!


In [47]:
###Reverse engineer empty Commit structure from rhino stream // GLOBALS

DL_modell = Collection(name = "DaylightModel", elements = [], collectionType = "Daylight Model")
DL_modell.elements.append(Collection(name = "Daylight_Inputs", elements = [], collectionType = "Layer",))
dl_Inputs = DL_modell.elements[0].elements

###Translate AC collection names to daylight layernames with the conversion dict

ac2Layers = [cTable[i.name] for i in AC_Collections if i.name in cTable.keys()]


###Filter geometry from the collections that made it through the dict
daylightGeo = [i for i in AC_Collections if i.name in cTable.keys()]

###create Collections from Daylight layernames
#for i in ac2Layers:
#    dl_Inputs.append(Collection(name = i, elements = [], collectionType = "layer"))


In [48]:

for i in range(0,len(daylightGeo)):
        
    ###add relevant collectionss to dl_imputs if not already present
    if cTable[daylightGeo[i].name] not in [col.name for col in dl_Inputs]:
        
        if cTable[daylightGeo[i].name].count("_") == 2 or daylightGeo[i].name == "Zone":
            dl_Inputs.append(Collection(name = cTable[daylightGeo[i].name], elements = [], collectionType = "layer" ))
            print (f"daylight_Inputs added {cTable[daylightGeo[i].name]} layer!") 
            targetIndex = [col.name for col in dl_Inputs].index(cTable[daylightGeo[i].name])
    ###iterate over relevant AC collections    
    for ele in daylightGeo[i].elements:
        
        ### handle regular geometry
        if daylightGeo[i].name not in ["Zone","CurtainWall"]:
 
            try:
                for dValue in ele.displayValue:
                    dValue.userStrings = createUstrings(ele)

                    dl_Inputs[targetIndex].elements.append(dValue)
            except:
                pass
        ### special case: zones
        elif daylightGeo[i].name == "Zone":
            failcount = 0
            try:
                ele.outline.userStrings = createUstrings(ele)
                dl_Inputs[targetIndex].elements.append(ele.outline)
            except:
                failcount +=1
    ### special case: curtain walls
    if daylightGeo[i].name == "CurtainWall":
        #Collections based on mesh.rendermaterial.opacity
        cWalls = daylightGeo[i].elements
        meshCollection = [*findGlass(cWalls).values()]
        print(meshCollection)
        sum = dl_Inputs.extend(meshCollection)
        


daylight_Inputs added _Walls_50 layer!
daylight_Inputs added _Roof_20 layer!
daylight_Inputs added _Columns_50 layer!
daylight_Inputs added _Floors_20 layer!
daylight_Inputs added _Curves layer!
[Collection(id: None, speckle_type: Speckle.Core.Models.Collection, totalChildrenCount: None), Collection(id: None, speckle_type: Speckle.Core.Models.Collection, totalChildrenCount: None)]
daylight_Inputs added _Context_30 layer!
daylight_Inputs added _Stairs_20 layer!
daylight_Inputs added _Railings_20 layer!


In [49]:

test1 = daylightGeo[5].elements
test2 = daylightGeo[5]
#TestVars = getChildren(test)  
#subEle = getChildren(daylightGeo[0].elements[:11])
#testCol = list(findGlass(test1).values())
#print (testCol)
#print (testCol.values())

In [50]:

walCols = [i for i in daylightGeo if i.name == "Wall"]
allWalls = []

for i in walCols:
    allWalls.extend(i.elements)

winDoors = getChildren(allWalls)
winDoorsByGlass = findGlass(winDoors)

#DL_modell.elements[0].elements += children.values()
DL_modell.elements[0].elements = dl_Inputs + [*winDoorsByGlass.values()]

print (DL_modell.elements[0].elements)

[Collection(id: None, speckle_type: Speckle.Core.Models.Collection, totalChildrenCount: None), Collection(id: None, speckle_type: Speckle.Core.Models.Collection, totalChildrenCount: None), Collection(id: None, speckle_type: Speckle.Core.Models.Collection, totalChildrenCount: None), Collection(id: None, speckle_type: Speckle.Core.Models.Collection, totalChildrenCount: None), Collection(id: None, speckle_type: Speckle.Core.Models.Collection, totalChildrenCount: None), Collection(id: None, speckle_type: Speckle.Core.Models.Collection, totalChildrenCount: None), Collection(id: None, speckle_type: Speckle.Core.Models.Collection, totalChildrenCount: None), Collection(id: None, speckle_type: Speckle.Core.Models.Collection, totalChildrenCount: None), Collection(id: None, speckle_type: Speckle.Core.Models.Collection, totalChildrenCount: None), Collection(id: None, speckle_type: Speckle.Core.Models.Collection, totalChildrenCount: None), Collection(id: None, speckle_type: Speckle.Core.Models.Coll

In [51]:
###send stream
#Send(DL_modell, "https://multiconsult.speckle.xyz/projects/8c1aca44f6")

In [52]:
def genDaylightModel(CommitData):
    
    AC_Collections = CommitData

    ###Reverse engineer empty Commit structure from rhino stream // GLOBALS

    DL_modell = Collection(name = "DaylightModel", elements = [], collectionType = "Daylight Model")
    DL_modell.elements.append(Collection(name = "Daylight_Inputs", elements = [], collectionType = "Layer",))
    dl_Inputs = DL_modell.elements[0].elements

    ###Filter geometry from the collections that made it through the dict
    daylightGeo = [i for i in AC_Collections if i.name in cTable.keys()]

    ###add objects to Daylight layers from AC type collections

    for i in range(0,len(daylightGeo)):
            
        ###add relevant collectionss to dl_imputs if not already present
        if cTable[daylightGeo[i].name] not in [col.name for col in dl_Inputs]:
            
            if cTable[daylightGeo[i].name].count("_") == 2 or daylightGeo[i].name == "Zone":
                dl_Inputs.append(Collection(name = cTable[daylightGeo[i].name], elements = [], collectionType = "layer" ))
                print (f"daylight_Inputs added {cTable[daylightGeo[i].name]} layer!") 
                targetIndex = [col.name for col in dl_Inputs].index(cTable[daylightGeo[i].name])
        ###iterate over relevant AC collections    
        for ele in daylightGeo[i].elements:
            
            ### handle regular geometry
            if daylightGeo[i].name not in ["Zone","CurtainWall"]:
    
                try:
                    for dValue in ele.displayValue:
                        dValue.userStrings = createUstrings(ele)

                        dl_Inputs[targetIndex].elements.append(dValue)
                except:
                    pass
            ### special case: zones
            elif daylightGeo[i].name == "Zone":
                failcount = 0
                try:
                    ele.outline.userStrings = createUstrings(ele)
                    dl_Inputs[targetIndex].elements.append(ele.outline)
                except:
                    failcount +=1
        ### special case: curtain walls
        if daylightGeo[i].name == "CurtainWall":
            #Collections based on mesh.rendermaterial.opacity
            cWalls = daylightGeo[i].elements
            meshCollection = [*findGlass(cWalls).values()]
            print(meshCollection)
            sum = dl_Inputs.extend(meshCollection)
            

    ###Get windows doors from walls
    walCols = [i for i in daylightGeo if i.name == "Wall"]
    allWalls = []
    for i in walCols:
        allWalls.extend(i.elements)

    winDoors = getChildren(allWalls)
    winDoorsByGlass = findGlass(winDoors)

    #DL_modell.elements[0].elements += children.values()
    DL_modell.elements[0].elements = dl_Inputs + [*winDoorsByGlass.values()]

    return DL_modell

In [ ]:
modelAB = genDaylightModel(AC_Collections)
print([i.name for i in modelAB.elements[0].elements])

In [20]:
from specklepy.api.wrapper import StreamWrapper
from specklepy.api.client import SpeckleClient
from specklepy.api import operations

from specklepy.objects import Base
from specklepy.objects.other import Collection
from specklepy.objects.geometry import Mesh
from specklepy.objects.other import RenderMaterial

Stream = "https://multiconsult.speckle.xyz/projects/e35c47291d/models/dc14e721c8"
#sWrap = StreamWrapper ("https://multiconsult.speckle.xyz/projects/8c1aca44f6")
sWrap = StreamWrapper(Stream)
sTransport = sWrap.get_transport()
sClient = sWrap.get_client()

dStream = sClient.stream.get(sWrap.stream_id)



b = dStream.branches
bb= dStream.branches.items
#print(b)
if "DaylightModels" in [i.name for i in bb]:
    dl_branch = sClient.branch.get("Daylight_Models", 1)
    print (dl_branch.name, dl_branch.id )
else:
    dl_branchID = sClient.branch.create(dStream.id ,"Daylight_Models",f"AC2Daylight via Python")
    print(dl_branchID)
   # client.branch.create("DaylightModels","daylight Modell source: ", urlStream)
#else: pass
#    dl_branch = client.branch.get("DaylightModels")
#   branchID = dl_branch.id"""



sendID = operations.send(DL_modell, [sTransport])
sClient.commit.create(sWrap.stream_id, sendID, branch_name="Daylight_Models",  message= " simple translate test" )

59e4250b9f
['main']


NameError: name 'DL_modell' is not defined